# Setup

Load the necessary modules from QGL. This will also import ``numpy`` into the namespace as ``np``. 

The AWGDir environment variable is used to indicate where QGL will store it's output sequence files. First we load the QGL module. It defaults to a temporary directory as provided by Python's tempfile module.


In [ ]:
from QGL import *

Next we instantiate the channel library. By default bbndb will use an sqlite database at the location specified by the BBN_DB environment variabe, but we override this behavior below in order to use a temporary in memory database for testing purposes.


In [ ]:
cl = ChannelLibrary(db_resource_name=":memory:")

The channel library has a number of convenience functions defined to create instruments and qubits, as well as functions to define the relationships between them. Let us create a qubit first:


In [ ]:
q1 = cl.new_qubit("q1")

In order to compile the QGL program into pulse sequences, we need to define a minimal hardware configuration. Basically, we need to specify AWG resources for output pulse compilation and digitizer resources for signal measurement.  

In [ ]:
# Most calls required label and address. Let's define 
# an AWG for control pulse generation 
aps2_1 = cl.new_APS2("BBNAPS1", address="192.168.5.101") 
# an AWG for measurement pulse generation
aps2_2 = cl.new_APS2("BBNAPS2", address="192.168.5.102")
# and digitizer for measurement collection
dig_1  = cl.new_X6("X6_1", address=0)

# Qubit q1 is controlled by AWG aps2_1, and uses microwave source h1
cl.set_control(q1, aps2_1)
# Qubit q1 is measured by AWG aps2_2 and digitizer dig_1, and uses microwave source h2
cl.set_measure(q1, aps2_2, dig_1.ch(1))

# Basic sequence construction and plotting

You can construct simple gate sequences by creating `Lists` of `Pulse` objects. These can be constructed by calling various primitives defined for qubits, for example, 90 and 180 degree rotations about X and Y:

In [15]:
seq1 = [[X(q1), Y(q1)]]
seq2 = [[X90(q1),Y90(q1),X(q1),Id(q1),Y(q1)]]


This sequence of pulses can be plotted for visual review. First, you must compile the QGL into pulses based on the hardware defined above. Since our `Qubit` object is a quadrature channel, you see two colors corresponding to the I and Q control signals.

In [16]:
mf = compile_to_hardware(seq1, 'Test1')

Compiled 1 sequences.


In [17]:
plot_pulse_files(mf)

Now, let's visualize the second sequence. 

In [18]:
mf = compile_to_hardware(seq2, 'Test2')
plot_pulse_files(mf)

Compiled 1 sequences.


# Constructing more sophisticated sequences (stopped here)

To get rotations of arbitrary angle (i.e. amplitude control of the pulse), you can use the "theta" primitives:

In [ ]:
seq = [Xtheta(q1, 0.2), Xtheta(q1, 0.4), Xtheta(q1, 0.6), Xtheta(q1, 0.8), Xtheta(q1, 1.0)]
show(seq)

To rotate about an arbitrary axis, use the "U" primitives:

In [ ]:
seq = [U(q1, 0.0), U(q1, np.pi/8), U(q1, np.pi/4), U(q1, 3*np.pi/8), U(q1, np.pi/2)]
show(seq)

Z rotations are done in "software", meaning that they act as frame changes on the following pulses.

In [ ]:
seq = [X(q1), Z90(q1), X(q1), Z90(q1), X(q1), Z90(q1), X(q1), Z90(q1), X(q1)]
show(seq)

Sequences can act on multiple channels. Let's load up another qubit channel.

In [ ]:
q2 = QubitFactory('q2')

When you `show` a sequence with multiple logical channels, each channel is plotted seperately.

In [ ]:
seq = [X(q1), X(q2), Y(q1), Y(q2)]
show(seq)

You express simultaneous operations with the `*` operator (meant to evoke a tensor product). If no operation is specified for a channel in a given time slot, an identity (no-op) operation is inserted.

In [ ]:
seq = [X(q1)*X(q2), X(q1)*Y(q2), Y(q1), X(q2)]
show(seq)

Measurements can be created with the `MEAS` primitive. Given a qubit X, they look up a logical measurement channel with the name M-X and create a `Pulse` on that channel, e.g.:

In [ ]:
seq = [MEAS(q1)]
show(seq)

In this case, the pulse is oscillating because it is an "autodyne" pulse (single source heterodyne) with an IF frequency of 10 MHz produced by single-sideband modulation. Normally single sideband modulation can be handled in a later step when a pulse sequence is compiled to hardware, but if the start of the pulse occurs at various times with respect to the start of the sequence, multiple modulated pulses will need to be generated with different phases. Since measurements tend to be somewhat long, this quickly consumes the available memory on the APS. Baking the SSB in before is a kludge used to conserve memory on the APS.

## Rabi amplitude

You can easily create sequences of pulses with list comprehensions. For instance, to vary a pulse amplitude in a Rabi sequence you might do something like:

In [ ]:
seqs = [[Xtheta(q1, a), MEAS(q1)] for a in np.linspace(0,1,11)]

In [ ]:
show(seqs[1])

In [ ]:
show(seqs[10])

## T1

In [ ]:
seqs = [[X(q1), Id(q1, d), MEAS(q1)] for d in np.linspace(0, 10e-6, 11)]

In [ ]:
show(seqs[0])

In [ ]:
show(seqs[1])

In [ ]:
show(seqs[2])

## Ramsey

In [ ]:
seqs = [[X90(q1), Id(q1, delay), X90(q1), MEAS(q1)] for delay in np.linspace(0, 5e-6, 11)]

In [ ]:
show(seqs[1])

In [ ]:
show(seqs[2])

## Compiling to hardware

When you are satisfied that you have constructed a sequence you would like to test, you can compile it to a sequence file that can be uploaded to the BBN APS.

In [ ]:
fileNames = compile_to_hardware(seqs, 'Ramsey')

In [ ]:
fileNames

There is an interactive plotter you may use to view the pulse sequence files.  From the notebook you can use an interactive slider to move through the sequences.

In [ ]:
plot_pulse_files(fileNames)